# Make an EPN-TAP data table for PLANMAP packages

We want to go inside Planmap packages and extract information for each data product in `vector` and `raster` directories.
This information will be used, initially, to populate VESPA EPN-TAP tables.
In particular, we're interested in filling the following columns<sup>*</sup>:

* `access_url`: data product URL
* `c1max`: max longitude (positive east, 0-360)
* `c1min`: min longitude (positive east, 0-360)
* `c2max`: max latitude
* `c2min`: min latitude
* `s_region`: map footprint
* `creation_date`: package creation date
* `dataproduct_type`: options are `im`
* `granule_gid`: package-id (PM_ID)
* `granule_uid`: data product name
* `modification_date`: package update date (same as creation data, if in doubt)
* `obs_id`: data product name
* `release_date`: package release date (same as creation date, if in doubt)
* `service_title`: `planmap`
* `spatial_frame_type`: `body`
* `target_class`: `planet`
* `target_name`: options are `mars`, `mercury`, `moon`
* `thumbnail_url`: thumbnail (jpeg) url

<sup>*</sup> About EPN-TAP columns: https://voparis-wiki.obspm.fr/display/VES/EPN-TAP+v2+parameter+description

### Software/libraries we're using here:

* Python 3.7.8
* rasterio 1.2.0
* shapely 1.7.1
* fiona 1.8.18

In [1]:
import pandas
pandas.set_option('display.max_colwidth',100)
pandas.set_option('display.max_rows', 100)

In [2]:
from dataclasses import make_dataclass, field, asdict

epntap_fields = [
    ('granule_uid', str),
    ('granule_gid', str),
    ('obs_id', str),
    ('c1min', float, field(default=0)),
    ('c1max', float, field(default=360)),
    ('c2min', float, field(default=-90)),
    ('c2max', float, field(default=90)),
    ('s_region', str, field(default='')),
    ('dataproduct_type', str, field(default='')),
    ('access_url', str, field(default='')),
    ('creation_date', str, field(default='2019-12-31')),
    ('modification_date', str, field(default='2019-12-31')),
    ('release_date', str, field(default='2019-12-31')),
    ('service_title', str, field(default='planmap')),
    ('spatial_frame_type', str, field(default='body')),
    ('target_class', str, field(default='planet')),
    ('target_name', str, field(default='')),
    ('thumbnail_url', str, field(default=''))
]

EPNTAPRecord = make_dataclass('EPNTAPRecord', epntap_fields)

EPNTAPRecordRaster = make_dataclass('EPNTAPRecordRaster', 
                                    [('dataproduct_type', str, 'im')],
                                    bases=(EPNTAPRecord,))

EPNTAPRecordVector = make_dataclass('EPNTAPRecordVector', 
                                    [('dataproduct_type', str, 'sv')],
                                    bases=(EPNTAPRecord,))

In [3]:
import pathlib

import fiona
import shapely
import rasterio


crs = {
    'moon': '+proj=longlat +a=1737400 +b=1737400 +no_defs',
    'mars': '+proj=longlat +a=3396190 +b=3376200 +no_defs',
    'mercury': '+proj=longlat +a=2439700 +b=2439700 +no_defs '
}

def bbox_shift_0360(bbox):
    c1min = bbox['c1min'] + 360 if bbox['c1min'] < 0 else bbox['c1min']
    c1max = bbox['c1max'] + 360 if bbox['c1max'] < 0 else bbox['c1max']
    bbox['c1min'] = c1min
    bbox['c1max'] = c1max
    return bbox
    
def bounding_box_raster(tif, body):
    from rasterio import warp
    img = rasterio.open(tif)
    dst_crs = crs[body]
    left, bottom, right, top = warp.transform_bounds(img.crs, dst_crs, *img.bounds)
    bbox = {
        'c1min': left,
        'c1max': right,
        'c2min': bottom,
        'c2max': top
    }
    return bbox_shift_0360(bbox)
    
def bounding_box_vector(gpkg, body):
    from rasterio import warp
    f = fiona.open(gpkg)
    dst_crs = crs[body]
    left, bottom, right, top = warp.transform_bounds(f.crs, dst_crs, *f.bounds)
    bbox = {
        'c1min': left,
        'c1max': right,
        'c2min': bottom,
        'c2max': top
    }
    return bbox_shift_0360(bbox)

def s_region_bbox(bbox):
    from shapely import geometry
    coords = [
        (bbox['c1min'],bbox['c2min']),
        (bbox['c1min'],bbox['c2max']),
        (bbox['c1max'],bbox['c2max']),
        (bbox['c1max'],bbox['c2min']),
    ]
    pol = geometry.Polygon(coords)
    return pol

def init_path(path):
    p = pathlib.Path(path)
    if not (p.exists() and p.is_dir()):
        return None
    return p
    
    
class PmPkg:
    def __init__(self, path):
        _path = init_path(path)
        assert _path is not None, "Path '{!s}' does not seem to exist".format(path)
        self._path = _path
        self._pmid = _path.name
        
    @property
    def pmid(self):
        return self._pmid
    
    def parse_readme(self, param):
        _files = self._path.glob('*.*')
        for _file in _files:
            if _file.suffix.lower() == '.md':
                break
        assert _file
        with open(_file, 'r') as fp:
            _lines = fp.readlines()
        for _l in _lines:
            _l = _l.strip().lower()
            if param.lower() in _l:
                return _l.split('|')[2].strip()
        return ''
        
    def proc_raster_dir(self):
        rasters = []
        path = self._path / 'raster'
        if path.exists and path.is_dir():
            for _f in path.iterdir():
                if _f.suffix == '.tif' or _f.suffix == '.tiff':
                    target_name = self.parse_readme('target body')
                    assert target_name in ('mars', 'mercury', 'moon'), "Expecting 'target_name' in 'mars', 'mercury', or 'moon'"
#                     thumbnail_url = '.'.join(access_url.split('.')[:-1] + ['jpg'])
                    bbox = bounding_box_raster(_f, target_name)
                    raster = self.make_epntap_record(_f, bbox, target_name)
                    raster['dataproduct_type'] = 'im'
                    rasters.append(raster)
        return rasters
        
    def proc_vector_dir(self):
        vectors = []
        path = self._path / 'vector'
        if path.exists and path.is_dir():
            for _f in path.iterdir():
                if _f.suffix != '.gpkg':
                    continue
                target_name = self.parse_readme('target body')
                assert target_name in ('mars', 'mercury', 'moon'), "Expecting 'target_name' in 'mars', 'mercury', or 'moon'"
                bbox = bounding_box_vector(_f, target_name)
                vector = self.make_epntap_record(_f, bbox, target_name)
                vector['dataproduct_type'] = 'sv'
                vectors.append(vector)
        return vectors                
    
    def proc_readme(self):
        def read_readme_table(readme_path):
            """
            Read README table into a dictionary
            """
            from string import ascii_letters
            out = {}
            with open(readme_path, 'r') as fp:
                lines = fp.readlines()
                for line in lines:
                    line = line.strip()
                    if len(line) == 0 or line[0] != '|':
                        continue
                    fields = line.strip('|').split('|')
                    if len(fields) < 2:
                        continue
                    k,v = [f.strip() for f in fields[:2]]
                    if k[0] in ascii_letters:
                        out[k] = v
            return out
        
        _files = self._path.glob('*.*')
        for _file in _files:
            if _file.suffix.lower() == '.md':
                break
        readme_table = read_readme_table(_file.as_posix())
        bbox = {
            'c1min' : float(readme_table['Bounding box - Min Lon (0-360)']),
            'c1max' : float(readme_table['Bounding box - Max Lon (0-360)']),
            'c2min' : float(readme_table['Bounding box - Min Lat']),
            'c2max' : float(readme_table['Bounding box - Max Lat'])
        }
        target_name = readme_table['Target body'].lower()
        _docs = _file.parent / 'document'
        file_map = _docs / (self.pmid + '.pdf')
        record = self.make_epntap_record(file_map, bbox, target_name)
        return record
        
    def make_epntap_record(self, file, bbox, target_name):
        granule_uid = file.stem
        granule_gid = self.pmid
        obs_id = granule_uid
        
        assert target_name in ('mars', 'mercury', 'moon'), """Expecting 'target_name' in 
                                                            'mars', 'mercury', or 'moon'"""
        
        access_url = '/'.join(['https://data.planmap.eu/pub', 
                               target_name, self.pmid, file.parent.name, file.name])

        record = dict(granule_uid = granule_uid,
                     obs_id = obs_id,
                     granule_gid = granule_gid,
                     access_url = access_url,
                     target_name = target_name,
                     c1min = bbox['c1min'],
                     c1max = bbox['c1max'],
                     c2min = bbox['c2min'],
                     c2max = bbox['c2max']
                    )
        return record
        
    def proc_data(self):
        records = []
        rasters = self.proc_raster_dir()
        records.extend(rasters)
        vectors = self.proc_vector_dir()
        records.extend(vectors)
        if not (rasters or vectors):
            readme = self.proc_readme()
            records.append(readme)
        self._records = records

    def to_epntap(self):
        return self._records
    

In [4]:
pkg = PmPkg('./PM-MAR-C-Arsinoes/')

pkg.proc_data()

epntap_records = pkg.to_epntap()

import pandas

pandas.DataFrame(epntap_records)

,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max,dataproduct_type
0,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_FEM,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_FEM,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,331.530619,331.758079,-13.169841,-12.980949,im
1,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_CHL,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_CHL,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,331.534151,331.759485,-13.169642,-12.980143,im
2,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_TRU,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_TRU,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,331.530619,331.758079,-13.169841,-12.980949,im
3,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_HYD,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_HYD,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,331.534151,331.759485,-13.169642,-12.980143,im
4,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_MAF,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_MAF,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,331.534151,331.759485,-13.169642,-12.980143,im


In [5]:
pkg = PmPkg('./PM-MAR-MS-Arsinoes/')

pkg.proc_data()

epntap_records = pkg.to_epntap()

import pandas

pandas.DataFrame(epntap_records)

,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max,dataproduct_type
0,PM-MAR-MS-Arsinoes,PM-MAR-MS-Arsinoes,PM-MAR-MS-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-MS-Arsinoes/raster/PM-MAR-MS-Arsinoes.tiff,mars,329.751644,334.431670,-12.003663,-5.749317,im
1,PM-MAR-MS-Arsinoes,PM-MAR-MS-Arsinoes,PM-MAR-MS-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-MS-Arsinoes/vector/PM-MAR-MS-Arsinoes.gpkg,mars,329.716783,334.516142,-12.084578,-5.815578,sv


In [6]:
pkg = PmPkg('./PM-MAR-D-Crommelin/')

pkg.proc_data()

epntap_records = pkg.to_epntap()

import pandas

pandas.DataFrame(epntap_records)

,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max
0,PM-MAR-D-Crommelin,PM-MAR-D-Crommelin,PM-MAR-D-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-D-Crommelin/document/PM-MAR-D-Crommelin.pdf,mars,348.8477,350.98821,3.96993,6.22761


In [7]:
import json

from IPython.display import display

import pathlib
cwd = pathlib.Path('.')

def summary_package(df,pmid):
    """
    This is a hack for planmap maps.planmap.eu app,
    I want to profit fom this processing and get the bounding-box in [-180:180] to put there.
    """
    if not len(df):
        return False
    name = '-'.join(pmid.split('-')[-2:])
    
    c1min = df.c1min.min()
    c1min = c1min if c1min < 180 else c1min - 360
    c1max = df.c1max.max()
    c1max = c1max if c1max < 180 else c1max - 360
    c2min = df.c2min.min()
    c2max = df.c2max.max()
    
    bla = {
        'name' : name,
        'pm_id' : pmid,
        'center' : dict(
            lon= (c1min + c1max)/2,
            lat= (c2min + c2max)/2
        ),
        'bbox' : dict(
            xmin= c1min,
            xmax= c1max,
            ymin= c2min,
            ymax= c2max,
            srs= 'EPSG:4326'
        )
    }
    return bla
    
    
df = None

packages = list(cwd.iterdir())
packages.sort()
for _dir in packages:
    if not (_dir.is_dir() and _dir.name[:2] == 'PM'):
        continue
        
    print('#---#---#')
    print(F"Package: {_dir}")
    try:
        pkg = PmPkg(_dir)
        pkg.proc_data()
        epntap_records = pkg.to_epntap()
    except Exception as err:
        print(err)
    _df = pandas.DataFrame(epntap_records)
    bla = summary_package(_df, _dir.name)
    if not bla:
        print("FAILED")
        continue
    print(json.dumps(bla, indent=2))
    display(_df)
    print('#---.---.')

    if df is None:
        df = _df
    else:
        df = pandas.concat([df,_df]).reset_index(drop=True)

#---#---#
Package: PM-MAR-C-Arsinoes
{
  "name": "C-Arsinoes",
  "pm_id": "PM-MAR-C-Arsinoes",
  "center": {
    "lon": -28.354948058817456,
    "lat": -13.0749921424451
  },
  "bbox": {
    "xmin": -28.469381174203818,
    "xmax": -28.240514943431094,
    "ymin": -13.169841214236444,
    "ymax": -12.980143070653755,
    "srs": "EPSG:4326"
  }
}


,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max,dataproduct_type
0,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_FEM,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_FEM,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,331.530619,331.758079,-13.169841,-12.980949,im
1,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_CHL,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_CHL,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,331.534151,331.759485,-13.169642,-12.980143,im
2,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_TRU,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_TRU,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,331.530619,331.758079,-13.169841,-12.980949,im
3,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_HYD,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_HYD,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,331.534151,331.759485,-13.169642,-12.980143,im
4,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_MAF,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_MAF,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,331.534151,331.759485,-13.169642,-12.980143,im


#---.---.
#---#---#
Package: PM-MAR-C-Crommelin
{
  "name": "C-Crommelin",
  "pm_id": "PM-MAR-C-Crommelin",
  "center": {
    "lon": -10.309911318473496,
    "lat": 7.279592356400499
  },
  "bbox": {
    "xmin": -10.844780922334394,
    "xmax": -9.775041714612598,
    "ymin": 4.39276139195179,
    "ymax": 10.166423320849209,
    "srs": "EPSG:4326"
  }
}


,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max,dataproduct_type
0,PM-MAR-C-Crommelin_01_FEM_frt0001fd59_07_if165s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_frt0001fd59_07_if165s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_frt0001fd59...,mars,349.389908,349.625871,9.959918,10.166423,im
1,PM-MAR-C-Crommelin_01_FEM_frs00029e82_01_if169s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_frs00029e82_01_if169s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_frs00029e82...,mars,349.176554,349.382756,10.038756,10.114374,im
2,PM-MAR-C-Crommelin_01_FEM_frs00029fb5_01_if167s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_frs00029fb5_01_if167s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_frs00029fb5...,mars,349.967445,350.224958,4.897460,5.001011,im
3,PM-MAR-C-Crommelin_01_frt0001fd59_07_if165s_trr3_TRU_st1-99,PM-MAR-C-Crommelin_01_frt0001fd59_07_if165s_trr3_TRU_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_frt0001fd59_07_...,mars,349.389908,349.625871,9.959918,10.166423,im
4,PM-MAR-C-Crommelin_01_hrl000064b2_07_if182s_trr3_TRU_st1-99,PM-MAR-C-Crommelin_01_hrl000064b2_07_if182s_trr3_TRU_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_hrl000064b2_07_...,mars,349.515848,349.777005,4.392761,4.800288,im
5,PM-MAR-C-Crommelin_01_frt00017c94_07_if165s_trr3_TRU_1-99,PM-MAR-C-Crommelin_01_frt00017c94_07_if165s_trr3_TRU_1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_frt00017c94_07_...,mars,349.155219,349.396637,4.668037,4.889413,im
6,PM-MAR-C-Crommelin_01_frs00029fb5_01_if167s_trr3_TRU_st1-99,PM-MAR-C-Crommelin_01_frs00029fb5_01_if167s_trr3_TRU_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_frs00029fb5_01_...,mars,349.967445,350.224958,4.897460,5.001011,im
7,PM-MAR-C-Crommelin_01_FEM_hrs00012a18_07_if174s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_hrs00012a18_07_if174s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_hrs00012a18...,mars,349.876231,350.084549,4.884428,5.039908,im
8,PM-MAR-C-Crommelin_01_hrs00012a18_07_if174s_trr3_TRU_st1-99,PM-MAR-C-Crommelin_01_hrs00012a18_07_if174s_trr3_TRU_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_hrs00012a18_07_...,mars,349.876231,350.084549,4.884428,5.039908,im
9,PM-MAR-C-Crommelin_01_FEM_hrl000064b2_07_if182s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_hrl000064b2_07_if182s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_hrl000064b2...,mars,349.515848,349.777005,4.392761,4.800288,im


#---.---.
#---#---#
Package: PM-MAR-D-Crommelin
{
  "name": "D-Crommelin",
  "pm_id": "PM-MAR-D-Crommelin",
  "center": {
    "lon": -10.082045000000022,
    "lat": 5.09877
  },
  "bbox": {
    "xmin": -11.152300000000025,
    "xmax": -9.011790000000019,
    "ymin": 3.96993,
    "ymax": 6.22761,
    "srs": "EPSG:4326"
  }
}


...

#---.---.
#---#---#
Package: PM-MOO-MS-Copernicus
CRS is invalid: None
{
  "name": "MS-Copernicus",
  "pm_id": "PM-MOO-MS-Copernicus",
  "center": {
    "lon": -23.0,
    "lat": -3.0
  },
  "bbox": {
    "xmin": -28.0,
    "xmax": -18.0,
    "ymin": -8.0,
    "ymax": 2.0,
    "srs": "EPSG:4326"
  }
}


,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max
0,PM-MOO-MS-Apollo12,PM-MOO-MS-Apollo12,PM-MOO-MS-Apollo12,https://data.planmap.eu/pub/moon/PM-MOO-MS-Apollo12/document/PM-MOO-MS-Apollo12.pdf,moon,332.0,342.0,-8.0,2.0


#---.---.
#---#---#
Package: PM-MOO-MS-SPAApollo
{
  "name": "MS-SPAApollo",
  "pm_id": "PM-MOO-MS-SPAApollo",
  "center": {
    "lon": -159.9104957597014,
    "lat": -34.17388959795346
  },
  "bbox": {
    "xmin": -179.45420444093048,
    "xmax": -140.3667870784723,
    "ymin": -61.710997049914084,
    "ymax": -6.636782145992844,
    "srs": "EPSG:4326"
  }
}


,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max,dataproduct_type
0,PM-MOO-MS-SPAApollo_LRO_LOLA_shaded,PM-MOO-MS-SPAApollo_LRO_LOLA_shaded,PM-MOO-MS-SPAApollo,https://data.planmap.eu/pub/moon/PM-MOO-MS-SPAApollo/raster/PM-MOO-MS-SPAApollo_LRO_LOLA_shaded.tif,moon,180.545796,179.729370,-61.710997,-6.636782,im
1,PM-MOO-MS-SPAApollo_LRO_LOLA,PM-MOO-MS-SPAApollo_LRO_LOLA,PM-MOO-MS-SPAApollo,https://data.planmap.eu/pub/moon/PM-MOO-MS-SPAApollo/raster/PM-MOO-MS-SPAApollo_LRO_LOLA.tif,moon,180.545796,179.729370,-61.710997,-6.636782,im
2,PM-MOO-MS-SPAApollo,PM-MOO-MS-SPAApollo,PM-MOO-MS-SPAApollo,https://data.planmap.eu/pub/moon/PM-MOO-MS-SPAApollo/vector/PM-MOO-MS-SPAApollo.gpkg,moon,195.917491,219.633213,-44.070362,-25.836114,sv


#---.---.


In [8]:
df

,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max,dataproduct_type
0,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_FEM,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_FEM,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,331.530619,331.758079,-13.169841,-12.980949,im
1,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_CHL,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_CHL,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,331.534151,331.759485,-13.169642,-12.980143,im
2,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_TRU,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_TRU,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,331.530619,331.758079,-13.169841,-12.980949,im
3,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_HYD,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_HYD,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,331.534151,331.759485,-13.169642,-12.980143,im
4,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_MAF,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_MAF,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,331.534151,331.759485,-13.169642,-12.980143,im
5,PM-MAR-C-Crommelin_01_FEM_frt0001fd59_07_if165s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_frt0001fd59_07_if165s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_frt0001fd59...,mars,349.389908,349.625871,9.959918,10.166423,im
6,PM-MAR-C-Crommelin_01_FEM_frs00029e82_01_if169s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_frs00029e82_01_if169s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_frs00029e82...,mars,349.176554,349.382756,10.038756,10.114374,im
7,PM-MAR-C-Crommelin_01_FEM_frs00029fb5_01_if167s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_frs00029fb5_01_if167s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_frs00029fb5...,mars,349.967445,350.224958,4.897460,5.001011,im
8,PM-MAR-C-Crommelin_01_frt0001fd59_07_if165s_trr3_TRU_st1-99,PM-MAR-C-Crommelin_01_frt0001fd59_07_if165s_trr3_TRU_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_frt0001fd59_07_...,mars,349.389908,349.625871,9.959918,10.166423,im
9,PM-MAR-C-Crommelin_01_hrl000064b2_07_if182s_trr3_TRU_st1-99,PM-MAR-C-Crommelin_01_hrl000064b2_07_if182s_trr3_TRU_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_hrl000064b2_07_...,mars,349.515848,349.777005,4.392761,4.800288,im


In [9]:
df.to_csv('/Users/chbrandt/Coisas/repos/VESPA/services/planmap/data/data.csv')

In [10]:
%ls

PM-MAR-C-Arsinoes@                  PM-MER-MS-H10_3cc@
PM-MAR-C-Crommelin@                 PM-MER-MS-H10_5cc@
PM-MAR-D-Crommelin@                 PM-MER-MS-H14_3cc@
PM-MAR-D-Gale_Kimberley@            PM-MER-MS-H14_5cc@
PM-MAR-D-Jezero@                    PM-MER-MS-Lermontov@
PM-MAR-D-Oxia_Planum@               PM-MER-MS-Rembrandt@
PM-MAR-MS-Arsinoes@                 PM-MER-SI-Rembrandt@
PM-MAR-MS-Crommelin@                PM-MOO-C-SPAA1_SU@
PM-MAR-SI-Crommelin_01@             PM-MOO-C-SPAApollo@
PM-MER-C-Beethoven@                 PM-MOO-C-SPAApollo_SU@
PM-MER-C-H05@                       PM-MOO-D-YutuGPR@
PM-MER-C-H05_SU@                    PM-MOO-GS-SPAApollo_Detail@
PM-MER-C-Rembrandt@                 PM-MOO-MS-Apollo11@
PM-MER-D-Rembrandt@                 PM-MOO-MS-Apollo12@
PM-MER-GS-Rachmaninoff_5cc@         PM-MOO-MS-Copernicus@
PM-MER-MS-Beethoven@                PM-MOO-MS-SPAApollo@
PM-MER-MS-H02_3cc@                  docs@
PM-MER-MS-H05_3cc@                  make_epntap_data

In [11]:
%ls ..

package_unwrap/     package_validator/  planmap-app-client/ storymaps/


In [12]:
packages

[PosixPath('.ipynb_checkpoints'),
 PosixPath('PM-MAR-C-Arsinoes'),
 PosixPath('PM-MAR-C-Crommelin'),
 PosixPath('PM-MAR-D-Crommelin'),
 PosixPath('PM-MAR-D-Gale_Kimberley'),
 PosixPath('PM-MAR-D-Jezero'),
 PosixPath('PM-MAR-D-Oxia_Planum'),
 PosixPath('PM-MAR-MS-Arsinoes'),
 PosixPath('PM-MAR-MS-Crommelin'),
 PosixPath('PM-MAR-SI-Crommelin_01'),
 PosixPath('PM-MER-C-Beethoven'),
 PosixPath('PM-MER-C-H05'),
 PosixPath('PM-MER-C-H05_SU'),
 PosixPath('PM-MER-C-Rembrandt'),
 PosixPath('PM-MER-D-Rembrandt'),
 PosixPath('PM-MER-GS-Rachmaninoff_5cc'),
 PosixPath('PM-MER-MS-Beethoven'),
 PosixPath('PM-MER-MS-H02_3cc'),
 PosixPath('PM-MER-MS-H05_3cc'),
 PosixPath('PM-MER-MS-H05_5cc'),
 PosixPath('PM-MER-MS-H10_3cc'),
 PosixPath('PM-MER-MS-H10_5cc'),
 PosixPath('PM-MER-MS-H14_3cc'),
 PosixPath('PM-MER-MS-H14_5cc'),
 PosixPath('PM-MER-MS-Lermontov'),
 PosixPath('PM-MER-MS-Rembrandt'),
 PosixPath('PM-MER-SI-Rembrandt'),
 PosixPath('PM-MOO-C-SPAA1_SU'),
 PosixPath('PM-MOO-C-SPAApollo'),
 PosixPath(

In [13]:

packages.sort()

In [14]:
packages

[PosixPath('.ipynb_checkpoints'),
 PosixPath('PM-MAR-C-Arsinoes'),
 PosixPath('PM-MAR-C-Crommelin'),
 PosixPath('PM-MAR-D-Crommelin'),
 PosixPath('PM-MAR-D-Gale_Kimberley'),
 PosixPath('PM-MAR-D-Jezero'),
 PosixPath('PM-MAR-D-Oxia_Planum'),
 PosixPath('PM-MAR-MS-Arsinoes'),
 PosixPath('PM-MAR-MS-Crommelin'),
 PosixPath('PM-MAR-SI-Crommelin_01'),
 PosixPath('PM-MER-C-Beethoven'),
 PosixPath('PM-MER-C-H05'),
 PosixPath('PM-MER-C-H05_SU'),
 PosixPath('PM-MER-C-Rembrandt'),
 PosixPath('PM-MER-D-Rembrandt'),
 PosixPath('PM-MER-GS-Rachmaninoff_5cc'),
 PosixPath('PM-MER-MS-Beethoven'),
 PosixPath('PM-MER-MS-H02_3cc'),
 PosixPath('PM-MER-MS-H05_3cc'),
 PosixPath('PM-MER-MS-H05_5cc'),
 PosixPath('PM-MER-MS-H10_3cc'),
 PosixPath('PM-MER-MS-H10_5cc'),
 PosixPath('PM-MER-MS-H14_3cc'),
 PosixPath('PM-MER-MS-H14_5cc'),
 PosixPath('PM-MER-MS-Lermontov'),
 PosixPath('PM-MER-MS-Rembrandt'),
 PosixPath('PM-MER-SI-Rembrandt'),
 PosixPath('PM-MOO-C-SPAA1_SU'),
 PosixPath('PM-MOO-C-SPAApollo'),
 PosixPath(